In [27]:
# CS340 Portfolio Project - Animal Shelter Dashboard
import dash
import dash_bootstrap_components as dbc
import vizro
import dash_daq as daq

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html, clientside_callback, Input, Output
import dash_auth
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Configure the AnimalShelter module to connect to the MongoDB database to handle CRUD operations
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Username and password for the MongoDB database not the web application
username = "aacuser"
password = "AACPASS9093"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)
# Types of Animals that will fit the critera for each type of service, includes a reset parameter.
rescue_types = ['Water Rescue', 'Mountain or Wilderness Rescue', 'Disaster or Individual Tracking',
                'Young Dogs', 'Old Dogs', 'Young Cats', 'Old Cats', 'Reset']

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

# Create a switch to change the color mode of the dashboard and place it in the top left corner.
# The switch will toggle between light and dark mode.
color_mode_switch = html.Span(
    [
        dbc.Label(className="fa fa-moon", html_for="color-mode-switch"),
        dbc.Switch( id="switch", value=True, className="d-inline-block ms-1", persistence=True),
        dbc.Label(className="fa fa-sun", html_for="color-mode-switch",)
    ]
)


app = dash.Dash(__name__, external_stylesheets=[vizro.bootstrap, dbc.icons.FONT_AWESOME, dbc_css],)

# set authentication for the dashboard app
#auth = dash_auth.BasicAuth(
#    app,
#    {
#        'admin': 'password'
#        }
#)


#Added Grazioso Salvare’s logo, scaled to fit the dashboard app
image_filename = 'Grazioso Salvare Logo Small.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())



app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    color_mode_switch,
    html.Center(html.B(html.H1('Animal Shelter Dashboard'))),
    html.Center(html.Img(src='data:image/png;base64, {}'.format(encoded_image.decode()))),
    html.Hr(),
    html.Center(html.B(html.H4('Locate animals using table below and find the best fit for your rescue needs.'))),
    html.Hr(),
    html.Div(className='dbc',
             style={'display': 'block'},
             children=[
             #Code for the interactive dropdown filtering menu.
             dcc.Dropdown(
                 id='filter-type',
                 options=[{'label': i, 'value': i} for i in rescue_types],
                 searchable=False,
                 placeholder='Select a rescue type',
             )]),
    html.Hr(),
    html.Div([
        dash_table.DataTable(id='datatable-id',
                            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                            data=df.to_dict('records'),
                            #Features for the interactive data table to make it user-friendly for your client
                            editable=False,
                            filter_action="native",
                            sort_action="native",
                            sort_mode="multi",
                            column_selectable=False,
                            row_selectable="single",
                            row_deletable=False,
                            selected_columns=[],
                            selected_rows=[0],
                            page_action="native",
                            page_current=0,
                            page_size=10

                        )
],
                className='dbc dbc-selectable',
                ),
    html.Div(
        [
            html.Button("Download Data as CSV", id="btn_csv"),
            dcc.Download(id="download-dataframe-csv"),
        ]
    ),

    html.Br(),
    html.Hr(),
             
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        
        ]),
    html.Br(),
    # Second Graph is set after a linebreak to place it under the previous graph.
    # Creates a histogram of the ages of the animals in the filtered category.
    html.Div(
            id='graph-id-2',
            className='col s12 m6'
            ),
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value',)]
              )
def update_dashboard(filter_type):

    ## Code to filter interactive data table with MongoDB queries

    dff = df

    if filter_type == 'Reset' or None:
        dff = df
    elif filter_type == 'Water Rescue':
        dff = df[df.breed.isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])
                              & (df.sex_upon_outcome == 'Intact Female')
                              & ((df.age_upon_outcome_in_weeks >=26) & (df.age_upon_outcome_in_weeks <= 156))]
    elif filter_type == 'Mountain or Wilderness Rescue':
        dff = df[df.breed.isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])
                              & (df.sex_upon_outcome == 'Intact Male')
                              & ((df.age_upon_outcome_in_weeks >= 26) & (df.age_upon_outcome_in_weeks <= 156))]
    elif filter_type == 'Disaster or Individual Tracking':
        dff = df[df.breed.isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])
                              & (df.sex_upon_outcome == 'Intact Male')
                              & ((df.age_upon_outcome_in_weeks >= 20) & (df.age_upon_outcome_in_weeks <= 300))]
    elif filter_type == 'Young Dogs':
        dff = df[df.animal_type.isin(['Dog'])
                                    & ((df.sex_upon_outcome == 'Intact Male') | (df.sex_upon_outcome == 'Intact Female'))
                                    & ((df.age_upon_outcome_in_weeks >= 12) & (df.age_upon_outcome_in_weeks <= 156))
                                    & (df.outcome_type != 'Euthanasia')]
    elif filter_type == 'Old Dogs':
        dff = df[df.animal_type.isin(['Dog'])
                                    & ((df.sex_upon_outcome == 'Intact Male') | (df.sex_upon_outcome == 'Intact Female'))
                                    & ((df.age_upon_outcome_in_weeks >= 300) & (df.age_upon_outcome_in_weeks <= 700))
                                    & (df.outcome_type != 'Euthanasia')]
    elif filter_type == 'Young Cats':
        dff = df[df.animal_type.isin(['Cat'])
                                    & ((df.sex_upon_outcome == 'Intact Male') | (df.sex_upon_outcome == 'Intact Female'))
                                    & ((df.age_upon_outcome_in_weeks >= 12) & (df.age_upon_outcome_in_weeks <= 156))
                                    & (df.outcome_type != 'Euthanasia')]
    elif filter_type == 'Old Cats':
        dff = df[df.animal_type.isin(['Cat'])
                                    & ((df.sex_upon_outcome == 'Intact Male') | (df.sex_upon_outcome == 'Intact Female'))
                                    & ((df.age_upon_outcome_in_weeks >= 300) & (df.age_upon_outcome_in_weeks <= 700))
                                    & (df.outcome_type != 'Euthanasia')]
   
   
    return dff.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    df_grouped = dff[['breed', 'animal_id']].groupby(['breed']).agg('count').reset_index()
    df_grouped = df_grouped.rename(columns = {'animal_id': 'Count'})
    
    fig = px.pie(df_grouped, values='Count', names='breed', title="Preferred Animal Types")
    return [
        dcc.Graph(figure=fig)
    ]
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    # Check if viewData or index is None or empty
    # Default coordinates and values to ensure the map is rendered correctly.
    if viewData is None or index is None or len(index) == 0:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(id="marker-id", position=[lat, lon], children=[
                    dl.Tooltip(breed),
                    dl.Popup([html.H3('Animal Name', style={'color': '#000000'}), # Text colors set to black to ensure visibility
                              html.P(name, style={'color': '#2C2C2C'})])
                ])
            ])
        ]
    
    # Convert viewData to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Get the selected row index
    row = index[0]
    
    # Ensure the selected row has valid latitude and longitude columns
    if 'location_lat' in dff.columns and 'location_long' in dff.columns:
        lat = dff.iloc[row]['location_lat']
        lon = dff.iloc[row]['location_long']
        breed = dff.iloc[row]['breed']
        name = dff.iloc[row]['name']


        # Return the updated map with the marker
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(id="marker-id", position=[lat, lon], children=[
                    dl.Tooltip(breed),
                    dl.Popup([html.H3('Animal Name', style={'color': '#000000'}),
                              html.P(name, style={'color': '#2C2C2C'})])
                ])
            ])
        ]
    else:
        # Fallback if latitude and longitude are missing
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(id="marker-id", position=[lat, lon], children=[
                    dl.Tooltip(breed),
                    dl.Popup([html.H3('Animal Name', style={'color': '#000000'}),
                              html.P(name, style={'color': "#2C2C2C"})])
                ])
            ])
        ]


    

# This callback updates a second graph to help show the distribution of animal ages in the filtered category
@app.callback(
    Output('graph-id-2', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_second_graph(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.histogram(dff, x="age_upon_outcome_in_weeks")
    return [
        dcc.Graph(figure=fig)
    ]


# This callback will download the data table as a CSV file
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("btn_csv", "n_clicks"),
    prevent_initial_call=True,
)
def func(n_clicks):
    return dcc.send_data_frame(df.to_csv, "animal_shelter_data.csv", index = False)

# This callback handles the color mode switch for the dashboard from light mode to dark mode and vice versa
clientside_callback(
    """ 
    (switchOn) => {
       document.documentElement.setAttribute('data-bs-theme', switchOn ? 'light' : 'dark');  
       return window.dash_clientside.no_update
    }
    """,
    Output("switch", "id"),
    Input("switch", "value"),
)

app.run(jupyter_mode="external", debug=False)


Dash app running on http://127.0.0.1:8050/


ERROR:__main__:Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\snowy\Desktop\CS 499 Artifacts to Enhance\CS340-Portfolio-Project-main\.venv\Lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\snowy\Desktop\CS 499 Artifacts to Enhance\CS340-Portfolio-Project-main\.venv\Lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\snowy\Desktop\CS 499 Artifacts to Enhance\CS340-Portfolio-Project-main\.venv\Lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\snowy\Desktop\CS 499 Artifacts to Enhance\CS340-Portfolio-Project-main\.venv\Lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~